In [1]:
import os
import numpy as np
import scipy.misc
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import SGD
from matplotlib.pyplot import imshow
import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

C:\Users\20173884\AppData\Local\Continuum\anaconda3\envs\8p361\lib\site-packages\tensorflow\python\keras\backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [2]:
def identity_block_l(X, f, filters, stage, block):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [3]:
def identity_block_s(X, f, filters, stage, block): 
    #DOES NOT WORK YET
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (f, f), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [4]:
def convolutional_block_l(X, f, filters, stage, block, s = 2):

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path 
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [5]:
def convolutional_block_s(X, f, filters, stage, block, s = 2):
    # DOES NOT WORK YET
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (f, f), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path 
    X = Conv2D(filters = F3, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [6]:
def ResNet12(input_shape=(96, 96, 3), classes=1):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    
    # Preprocess input to the correct size
    #X = Resizing(64,64)

    # Zero-Padding
    X = ZeroPadding2D((3,3))(X_input)

    # Stage 1
    X = Conv2D(64, (7,7), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides=(2,2))(X)
    
    # Stage 2
    X = convolutional_block_s(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block_s(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block_s(X, 3, [64, 64, 256], stage=2, block='c')
    
    # Stage 3
    X = convolutional_block_s(X, f=3, filters=[128,128,512], stage=3, block='a', s=2)
    X = identity_block_s(X, 3, [128,128,512], stage=3, block='b')

    # AVGPOOL
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet12')
    model.compile(SGD(lr=0.01, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

In [7]:
def ResNet24(input_shape=(96, 96, 3), classes=1):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block_s(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block_s(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block_s(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block_s(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block_s(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block_s(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block_s(X, 3, [128, 128, 512], stage=3, block='d')
    X = identity_block_s(X, 3, [128, 128, 512], stage=3, block='e')
    
    # Stage 4
    X = convolutional_block_s(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block_s(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block_s(X, 3, [256, 256, 1024], stage=4, block='c')

    # AVGPOOL
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet24')
    model.compile(SGD(lr=0.01, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

In [8]:
def ResNet40(input_shape=(96, 96, 3), classes=1):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1 (1 layer)
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block_s(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block_s(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block_s(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 (12 layers)
    X = convolutional_block_s(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block_s(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block_s(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block_s(X, 3, [128, 128, 512], stage=3, block='d')
    X = identity_block_s(X, 3, [128, 128, 512], stage=3, block='e')
    X = identity_block_s(X, 3, [128, 128, 512], stage=3, block='f')
    
    # Stage 4 (14 layers)
    X = convolutional_block_s(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block_s(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block_s(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block_s(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block_s(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block_s(X, 3, [256, 256, 1024], stage=4, block='f')
    X = identity_block_s(X, 3, [256, 256, 1024], stage=4, block='g')
    
    # Stage 5 (6 layers)
    X = convolutional_block_s(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block_s(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block_s(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    # output layer (1 layer)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet40')
    model.compile(SGD(lr=0.01, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

In [9]:
def ResNet50(input_shape=(96, 96, 3), classes=1):
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block_l(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block_l(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block_l(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block_l(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block_l(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block_l(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block_l(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block_l(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block_l(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block_l(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block_l(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block_l(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block_l(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block_l(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block_l(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block_l(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    model.compile(SGD(lr=0.01, momentum=0.95), loss = 'binary_crossentropy', metrics=['accuracy'])

    return model